# Q&A with RAG

## Pre-requisites

In [1]:
# Import libraries
import os
import sys
import shutil

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, "..")) # absolute path for ekr_rag directory
repo_dir = os.path.abspath(os.path.join(kit_dir, "..")) # absolute path for starter-kit directory
print('kit_dir: %s'%kit_dir)
print('repo_dir: %s'%repo_dir)

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.document_retrieval import DocumentRetrieval
from utils.parsing.sambaparse import SambaParse, parse_doc_universal

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')
PERSIST_DIRECTORY = os.path.join(kit_dir,f"data/my-vector-db")

kit_dir: /Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever
repo_dir: /Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit


/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/ekr_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Document loading and splitting

In [2]:
# Specify PDF file
#pdf_file = kit_dir + '/data/tmp/SN40LPaper2Pages.pdf'

# Initialize DocumentRetrieval class
documentRetrieval =  DocumentRetrieval()

# Parse the documents
additional_metadata = {}
_, _, text_chunks = parse_doc_universal(doc=kit_dir+'/data/test', additional_metadata=additional_metadata)

# Get the text chunks
#text_chunks = documentRetrieval.get_text_chunks_with_metadata(docs=raw_text, meta_data=meta_data) # lst of langchain_core.documents.base.Document
#print(text_chunks[0].page_content)
print('Nb of chunks: %d'%len(text_chunks))


2024-07-11 12:58:27,396 [INFO] - Deleting contents of output directory: ./output
2024-07-11 12:58:27,426 [INFO] - Running command: unstructured-ingest local --output-dir ./output --num-processes 2 --strategy auto --ocr-languages eng --encoding utf-8 --fields-include element_id,text,type,metadata,embeddings --metadata-exclude  --metadata-include  --pdf-infer-table-structure --input-path "/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/data/test" --recursive --verbose --chunking-strategy by_title --chunk-max-characters 1500 --chunk-overlap 300 --chunk-combine-text-under-n-chars 1500 --chunk-elements
2024-07-11 12:58:27,429 [INFO] - This may take some time depending on the size of your data. Please be patient...
/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/ekr_venv/lib/python3.10/site-packages/dataclasses_json/core.py:201: RuntimeWarning: 'NoneType' object value of non-optional type 

Nb of chunks: 10


## Vectorization and storage

In [3]:
# Create vector store
embeddings = documentRetrieval.load_embedding_model()
if os.path.exists(PERSIST_DIRECTORY):
    shutil.rmtree(PERSIST_DIRECTORY)
    print(f"The directory Chroma has been deleted.")
#vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=None)
vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=PERSIST_DIRECTORY)

/Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/ekr_venv/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
2024-07-11 12:59:33,637 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-07-11 12:59:40,562 [INFO] - Use pytorch device: cpu


max_seq_length  512
The directory Chroma has been deleted.


2024-07-11 12:59:44,250 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-11 12:59:58,923 [INFO] - Vector store saved to /Users/petrojm/Documents/projects/ASK/ekr_refactoring/ai-starter-kit/enterprise_knowledge_retriever/data/my-vector-db


## Retrieval and generation

In [4]:
# Create conversation chain
documentRetrieval.init_retriever(vectorstore)
conversation = documentRetrieval.get_qa_retrieval_chain()

In [5]:
# Ask questions about your data
user_question = "What is a composition of experts?"

response = conversation.invoke({"question":user_question})
print(response['question'])
print(response['answer'])
#print(len(response['source_documents']))

for i in range(0,len(response['source_documents'])):
    print('\nSource #%d:'%(i+1))
    print(response['source_documents'][i].page_content)
    print(response['source_documents'][i].metadata)

ValueError: Error raised by the inference endpoint: Error raised by the inference endpoint: Sambanova /complete call failed with status code 403.
Message: The API Key provided is either incorrect or revoked
Details: []
Code: 7
